In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import seaborn as sns
from folium.plugins import HeatMap, MarkerCluster
import networkx as nx
import time

# Load a sample geospatial dataset
# Assuming a CSV file with 'latitude' and 'longitude' columns
data_path = "your_dataseadasfaast.csv"  # Replace with actual dataset path
df = pd.read_csv(data_path)

# Convert to GeoDataFrame
geometry = gpd.points_from_xy(df['longitude'], df['latitude'])
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Question 1: Geospatial Visualization
# 1. Choropleth Map
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
fig, ax = plt.subplots(figsize=(10, 6))
world.boundary.plot(ax=ax, linewidth=1)
gdf.plot(ax=ax, markersize=10, color='red', alpha=0.5)
plt.title("Choropleth Map of Locations")
plt.show()

# 2. Bubble Map
fig, ax = plt.subplots(figsize=(10, 6))
world.boundary.plot(ax=ax, linewidth=1)
gdf.plot(ax=ax, markersize=df['some_value_column']*10, color='blue', alpha=0.5)
plt.title("Bubble Map Representation")
plt.show()

# 3. Hexagonal Binning
fig, ax = plt.subplots(figsize=(10, 6))
hexbin = ax.hexbin(df['longitude'], df['latitude'], gridsize=50, cmap='Blues', mincnt=1)
cbar = plt.colorbar(hexbin)
cbar.set_label("Frequency")
plt.title("Hexagonal Binning of Data Points")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

# 4. Heat Map using Folium
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=10)
heat_data = [[row['latitude'], row['longitude']] for _, row in df.iterrows()]
HeatMap(heat_data).add_to(m)
m.save("heatmap.html")
print("Heat map saved as heatmap.html")

# 5. Cluster Map
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=10)
marker_cluster = MarkerCluster().add_to(m)
for _, row in df.iterrows():
    folium.Marker([row['latitude'], row['longitude']]).add_to(marker_cluster)
m.save("clustermap.html")
print("Cluster map saved as clustermap.html")

# Question 2: Adding Points of Interest in Saudi Arabia
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=10)
poi = [(24.7136, 46.6753, "Riyadh"), (21.3891, 39.8579, "Jeddah")]
for lat, lon, name in poi:
    folium.Marker([lat, lon], popup=name, icon=folium.Icon(color='red')).add_to(m)
m.save("poi_map.html")
print("Map with points of interest saved as poi_map.html")

# Question 3: Routing Between Two Points using Different Algorithms
def create_graph():
    G = nx.Graph()
    for _, row in df.iterrows():
        G.add_node((row['latitude'], row['longitude']))
    return G

def compute_route(G, start, end, algorithm):
    if algorithm == 'bfs':
        return nx.shortest_path(G, start, end, method='bfs')
    elif algorithm == 'dfs':
        return nx.dfs_tree(G, start).nodes()
    elif algorithm == 'dijkstra':
        return nx.shortest_path(G, start, end, weight='length')
    else:
        return None

G = create_graph()
start, end = poi[0], poi[1]
routes = {}

for alg in ['bfs', 'dfs', 'dijkstra']:
    start_time = time.time()
    route = compute_route(G, start, end, alg)
    end_time = time.time()
    routes[alg] = {'route': route, 'time': end_time - start_time}
    print(f"{alg.upper()} completed in {end_time - start_time} seconds")

# Markdown Observations (to be included in the Jupyter Notebook)
md_observations = """
# Observations
1. The **Choropleth Map** helps visualize the distribution of locations over the world map.
2. The **Bubble Map** provides an indication of the magnitude of a particular attribute (e.g., accident severity).
3. **Hexagonal Binning** is useful for identifying dense clusters of points in an aggregated way.
4. The **Heat Map** highlights the most frequent locations, making hotspots clear.
5. The **Cluster Map** dynamically groups points to show high-density areas effectively.
6. The added **points of interest (Riyadh and Jeddah)** allow us to highlight important locations for further analysis.
7. Routing algorithms such as **BFS, DFS, and Dijkstra** vary in efficiency:
   - **BFS** is effective in finding the shortest path in an unweighted graph.
   - **DFS** explores as far as possible before backtracking, leading to inefficiencies.
   - **Dijkstra** provides an optimal shortest path but is computationally expensive.
"""


## hiiiiiiiiiiiiii